<a href="https://colab.research.google.com/github/danbom/tensorflow/blob/master/Lab07_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab 07-3 linear regression - Eager(normalization, decay, l2_loss)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

tf.random.set_seed(777)

- 정규화를 위한 함수 ( 최대 최소값이 1과 0이 되도록 Scaling 한다 )

In [ ]:
def normalization(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / denominator

In [ ]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

x_train = xy[:,0:-1]
y_train = xy[:,[-1]]

plt.plot(x_train, 'ro')
plt.plot(y_train)
plt.show()

- 표준화 적용해 실행

In [ ]:
xy = normalization(xy)
print(xy)
x_train = xy[:,0:-1]
y_train = xy[:,[-1]]

plt.plot(x_train, 'ro')
plt.plot(y_train)

plt.show()

- 위 Data 기준으로 Linear Regression 모델 만들기

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

W = tf.Variable(tf.random.normal((4,1)),dtype=tf.float32)
b = tf.Variable(tf.random.normal((1,)), dtype=tf.float32)

- Linear Regression Hypothesis 정의 (y = Wx + b)

In [ ]:
def linearReg_fn(features):
    hypothesis = tf.matmul(features, W) + b
    return hypothesis

- L2 loss를 적용할 함수 정의
-- Weight의 수가 많아지면 수만큼 더한다 tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3)

In [ ]:
def l2_loss(loss, beta=0.01):
    W_reg = tf.nn.l2_loss(W) # output = sum(t**2) / 2
    loss = tf.reduce_mean(loss + W_reg * beta)
    return loss

- 가설을 검증할 Cost 함수 정의 (Mean Square Error 사용)

In [ ]:
def loss_fn(hypothesis, features, labels, flag = False):
    cost = tf.reduce_mean(tf.square(hypothesis - labels))
    if (flag):
        cost = l2_loss(cost)
    return cost

- Learning Decay 설정

In [ ]:
is_decay = True
starter_learning_rate = 0.1

if(is_decay):
    learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=starter_learning_rate, decay_rate=0.9, decay_steps=50, staircase=True)
    optimizer = tf.keras.optimizers.SGD(learning_rate)

else:
    optimizer = tf.keras.optimizers.SGD(learning_rate=starter_learning_rate)

def grad(hypothesis, features, labels, l2_flag):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(linearReg_fn(features), features, labels, l2_flag)
    return tape.gradient(loss_value, [W,b]), loss_value

In [ ]:
EPOCHS = 101

for step in range(EPOCHS):
    for features, labels in dataset:
        features = tf.cast(features, tf.float32)
        labels = tf.cast(labels, tf.float32)
        grads, loss_value = grad(linearReg_fn(features), features, labels, False)
        optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
    if step % 10 == 0:
        print("Iter:{}, Loss:{:4f}".format(step, loss_value))